In [1]:
import boto3
import pandas as pd
from io import BytesIO
from datetime import datetime

endpoint_url =  'http://localhost:9000'
aws_access_key_id = 'teste'
aws_secret_access_key = 'OH9PTqKV1FRWV5fmWAMAEfUh05JWR1TOESxIhMww'
bucket_name = 'tiradentes'
filename = 'revenue/tiradentes-revenue-2024-01-24'

s3_resource = boto3.resource(
    's3', 
    endpoint_url=endpoint_url,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    aws_session_token=None,
)

bucket = s3_resource.Bucket(bucket_name)
    
obj = s3_resource.Object(bucket_name, filename)
df = pd.read_parquet(BytesIO(obj.get()['Body'].read()))

In [5]:
SELECT_LAST_REVENUE_DATE_FROM_MONTH = """
    select date
    from revenue
    where to_char(date, 'YYYY-MM') = '{}'
    order by date desc
    limit 1
"""

CITY_REVENUE_COLUMNS_RENAME = {
    'Data': 'date',
    'Tipo de Minuta': 'draft_type',
    'Receita': 'revenue',
    'Unnamed: 3': 'source',
    'Fonte de Recursos': 'resource_source',
    'CO TCE': 'co_tce',
    'CO AUX': 'co_aux',
    'Histórico': 'historic',
    'Valor': 'value',
}
df = pd.read_parquet(BytesIO(obj.get()['Body'].read()))


city_name = 'tiradentes'

df.head()

# df.query('not Data.str.contains("TOTAL")', engine='python', inplace=True)
# df.query('not Data.str.contains("Total do Dia")', engine='python', inplace=True)
# df.rename(columns=CITY_REVENUE_COLUMNS_RENAME, inplace=True)

# df['date'] = pd.to_datetime(df['date'], format="%d/%m/%Y")
# df['city'] = city_name


# query_cursor = postgres_resource.execute_query(
#     SELECT_LAST_REVENUE_DATE_FROM_MONTH.format(partition_date_str[:7])
# )

# last_revenue = False
# try:
#     last_revenue = query_cursor.one()
# except NoResultFound:
#     context.log.info("No record of previous revenue found in database")
# if last_revenue:
#     last_revenue_date = last_revenue[0]
#     df.query(f'date > "{last_revenue_date}"', inplace=True)

df



,Data,Tipo de Minuta,Receita,Unnamed: 3,Fonte de Recursos,CO TCE,CO AUX,Histórico,Valor
0,TOTAL,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
import pandas as pd
from sqlalchemy import create_engine, URL, text

url = URL.create(
    "postgresql+psycopg2",
    username='postgres',
    password='postgres',
    host='localhost',
    port=5432,
    database='gov_transparency_hub',
)

conn = create_engine(url).connect()

df.to_sql('revenue', conn, if_exists='append', index=False)

473